In [87]:
import pandas as pd
true_label = pd.read_csv("./responses-carb.csv")
true_label = true_label["class"]
true_label = true_label.tolist()

In [88]:
import numpy as np
from scipy.stats import mode

# Assuming you have an array of 500 numbers stored in the variable 'numbers'
def get_accuracy(numbers):
    chunk_size = 100
    num_chunks = len(numbers) // chunk_size

    modes = []

    for i in range(num_chunks):
        chunk = numbers[i * chunk_size: (i + 1) * chunk_size]
        mode_result = mode(chunk)
        mode_value = mode_result.mode
        mode_count = mode_result.count
        modes.append((mode_value, mode_count))

    print(modes)

In [89]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
data = pd.read_csv('similarity-matrix-k=3.csv')
similarity_matrix = data.drop(columns=['Unnamed: 0']) 
k = 2  # Number of clusters
kmeans = KMeans(n_clusters=2, random_state=0)
kmeans.fit(similarity_matrix)
kmeans_labels = np.array(kmeans.labels_)
# kmeans_labels

/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [90]:
from sklearn.mixture import GaussianMixture as GMM
gmm = GMM(n_components=2, random_state=0).fit(similarity_matrix)
gmm_labels = gmm.predict(similarity_matrix)
# gmm_labels

/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/base.py:1151: ConvergenceWarning: Number of distinct clusters (1) found smaller than n_clusters (2). Possibly due to duplicate points in X.
  return fit_method(estimator, *args, **kwargs)


In [91]:
from sklearn.cluster import SpectralClustering
spectral_clustering = SpectralClustering(n_clusters=2,random_state=0)
spectral_clustering.fit(similarity_matrix)
spectral_labels = spectral_clustering.labels_
spectral_labels

/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/cluster/_spectral.py:688: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn(


array([1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0,
       1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1], dtype=int32)

In [92]:
get_accuracy(kmeans_labels)

[(0, 100)]


In [93]:
get_accuracy(gmm_labels)

[(0, 100)]


In [94]:
get_accuracy(spectral_labels)

[(1, 51)]


In [95]:
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import csv

# Assuming you have a similarity matrix stored in the variable 'similarity_matrix'
# Create an instance of PCA
pca = PCA(n_components=3) 

# Apply PCA on the similarity matrix
principal_components = pca.fit_transform(similarity_matrix)

# Get the PC1 and PC2 values
pc1 = principal_components[:, 0]
pc2 = principal_components[:, 1]
pc3 = principal_components[:, 2]

column_names = ['true_label', 'pc1', 'pc2', 'gmm_result', 'kmeans_result', 'spectral_result']
rows = zip(true_label, pc1, pc2, gmm_labels, kmeans_labels, spectral_labels)

# Specify the file name
filename = 'bingo3c_result.csv'

# Write the data to the CSV file
with open(filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    
    # Write the column names
    writer.writerow(column_names)
    
    # Write the data rows
    writer.writerows(rows)

/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/decomposition/_pca.py:543: RuntimeWarning: invalid value encountered in divide
  explained_variance_ratio_ = explained_variance_ / total_var


In [96]:
verify = pd.read_csv('bingo3c_result.csv')
print(sum(verify['true_label'] == verify['kmeans_result'])/len(verify['true_label']))
print(sum(verify['true_label'] == verify['gmm_result'])/len(verify['true_label']))
print(sum(verify['true_label'] == verify['spectral_result'])/len(verify['true_label']))

us_kmeans = pd.read_csv('kmeans_unsup_predictions.csv')
#print(sum(verify['kmeans_result'] == us['Labels'])/len(verify['kmeans_result']))

us_gmm = pd.read_csv('gmm_unsup_predictions.csv')
#print(sum(verify['gmm_result'] == us_gmm['Labels'])/len(verify['gmm_result']))

us_spec = pd.read_csv('spectral_unsup_predictions.csv')
#print(sum(verify['spectral_result'] == us_spec['Labels'])/len(verify['spectral_result']))

0.7950819672131147
0.7950819672131147
0.3770491803278688


In [97]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [110]:
print("------K-means-----------")
print("& BioKlustering & Bingo-3C \\\\\\hline")
print(F"False positive rate & {1 - precision_score(verify['true_label'], us_kmeans['Labels'],pos_label=1)} & {1 - precision_score(verify['true_label'], verify['kmeans_result'],pos_label=1)}\\\\")
print(F"False negative rate & {1 - precision_score(verify['true_label'], us_kmeans['Labels'],pos_label=0)} & {1 - precision_score(verify['true_label'], verify['kmeans_result'],pos_label=0)}")

print("------GMM---------")
print("& BioKlustering & Bingo-3C \\\\\\hline")
print(F"False positive rate & {1 - precision_score(verify['true_label'], us_gmm['Labels'],pos_label=1)} & {1 - precision_score(verify['true_label'], verify['gmm_result'],pos_label=1)}\\\\")
print(F"False negative rate & {1 - precision_score(verify['true_label'], us_gmm['Labels'],pos_label=0)} & {1 - precision_score(verify['true_label'], verify['gmm_result'],pos_label=0)}")


print("-------Spectral---------")
print("& BioKlustering & Bingo-3C \\\\\\hline")
print(F"False positive rate & {1 - precision_score(verify['true_label'], us_spec['Labels'],pos_label=1)} & {1 - precision_score(verify['true_label'], verify['spectral_result'],pos_label=1)}\\\\")
print(F"False negative rate & {1 - precision_score(verify['true_label'], us_spec['Labels'],pos_label=0)} & {1 - precision_score(verify['true_label'], verify['spectral_result'],pos_label=0)}")

------K-means-----------
& BioKlustering & Bingo-3C \\\hline
False positive rate & 0.7767857142857143 & 1.0\\
False negative rate & 0.0 & 0.20491803278688525
------GMM---------
& BioKlustering & Bingo-3C \\\hline
False positive rate & 0.0 & 1.0\\
False negative rate & 0.18487394957983194 & 0.20491803278688525
-------Spectral---------
& BioKlustering & Bingo-3C \\\hline
False positive rate & 0.782608695652174 & 0.8805970149253731\\
False negative rate & 0.19736842105263153 & 0.3090909090909091


/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# SEMI-SUPERVISED

In [108]:
us_kmeans = pd.read_csv('kmeans_semisup_predictions.csv')
us_gmm = pd.read_csv('gmm_semisup_predictions.csv')
us_spec = pd.read_csv('spectral_semisup_predictions.csv')

print("SEMISUPERVISED")

SEMISUPERVISED


In [109]:
print("------K-means-----------")
print("& BioKlustering & Bingo-3C \\\\\\hline")
print(F"False positive rate & {precision_score(verify['true_label'], us_kmeans['Labels'],pos_label=1)} & {precision_score(verify['true_label'], verify['kmeans_result'],pos_label=1)}\\\\")
print(F"False negative rate & {precision_score(verify['true_label'], us_kmeans['Labels'],pos_label=0)} & {precision_score(verify['true_label'], verify['kmeans_result'],pos_label=0)}")

print("------GMM---------")
print("& BioKlustering & Bingo-3C \\\\\\hline")
print(F"False positive rate & {precision_score(verify['true_label'], us_gmm['Labels'],pos_label=1)} & {precision_score(verify['true_label'], verify['gmm_result'],pos_label=1)}\\\\")
print(F"False negative rate & {precision_score(verify['true_label'], us_gmm['Labels'],pos_label=0)} & {precision_score(verify['true_label'], verify['gmm_result'],pos_label=0)}")


print("-------Spectral---------")
print("& BioKlustering & Bingo-3C \\\\\\hline")
print(F"False positive rate & {precision_score(verify['true_label'], us_spec['Labels'],pos_label=1)} & {precision_score(verify['true_label'], verify['spectral_result'],pos_label=1)}\\\\")
print(F"False negative rate & {precision_score(verify['true_label'], us_spec['Labels'],pos_label=0)} & {precision_score(verify['true_label'], verify['spectral_result'],pos_label=0)}")

------K-means-----------
& BioKlustering & Bingo-3C \\\hline
False positive rate & 0.22321428571428573 & 0.0\\
False negative rate & 1.0 & 0.7950819672131147
------GMM---------
& BioKlustering & Bingo-3C \\\hline
False positive rate & 1.0 & 0.0\\
False negative rate & 0.8151260504201681 & 0.7950819672131147
-------Spectral---------
& BioKlustering & Bingo-3C \\\hline
False positive rate & 0.21739130434782608 & 0.11940298507462686\\
False negative rate & 0.8026315789473685 & 0.6909090909090909


/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/samoz/Documents/masters/research/yuke-bioklustering/bioklustering/.conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
